# Transforma arquivos do Tabnet
Como usar:

* Acessar http://tabnet.datasus.gov.br/cgi/deftohtm.exe?sih/cnv/nrbr.def
* Escolher
    - Linha: Município
    - Conteúdo: Internações
    - Período disponível: Selecionar de Jan/2017 a Dez/2017
    - Lista Morb CID-10: Selecionar doença (ex: traumatismo intracraniano)
* Escolher opção "colunas separadas por ";""
* Clicar em Mostra
* Copiar dados a partir da linha "Município";"Internações" até a ultima linha (desconsiderando a linha "Total";xxx)
* Criar um arquivo .csv nessa pasta com o nome da doença em questão (ex: traumatismo.csv)
* Atualizar o nome do arquivo e da doença no código abaixo
* Rodar o código

Import libraries

In [ ]:
import pandas as pd

Creates quantity file

In [30]:
doenca = 'TRAUMATISMO'
table = pd.read_csv('traumatismo.csv', sep=';')
table['MUNCOD'] = table['Município'].str.split(' ', expand=True,)[0]
table.drop('Município', axis=1, inplace=True)
table.rename(columns={"Internações": "QUANTIDADE"}, inplace=True)
table.to_csv('Quantity/' + doenca + 'BR2017.csv')

Creates rates file

In [31]:
table = pd.read_csv('Quantity/' + doenca + 'BR2017.csv')
df=pd.read_csv('../../util/POPULATION_2017.csv')
suicide = pd.read_csv('../../Suicide/CSV/suicide_rate_codmunres.csv')
result = pd.merge(table, df, left_on="MUNCOD", right_on="MUNCOD", how="inner")
result = pd.merge(result, suicide, left_on="MUNCOD", right_on="CODMUNRES", how="inner")
result['QUANTIDADE_RATE'] = result['QUANTIDADE']/result['POPULATION'] * 100000
result = result[["QUANTIDADE_RATE", "SUICIDE_RATE", "MUNCOD"]]
result.to_csv('Rates/' + doenca + "BR2017_RATE.csv")